In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.26.0.3  70.25 KiB  16      68.2%             81b4a457-e16a-4a02-814d-f512b8a65a9d  rack1
UN  172.26.0.2  70.26 KiB  16      70.2%             3f9ad223-fdfc-4c9b-a8cd-0750da95d497  rack1
UN  172.26.0.4  70.23 KiB  16      61.6%             35f18065-50a7-4204-ae3c-847acea9bf88  rack1



In [8]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [9]:
cass.execute("DROP KEYSPACE IF EXISTS weather")

In [10]:
cass.execute("""
CREATE KEYSPACE IF NOT EXISTS weather
WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3 
};
""")

In [19]:
cass.execute("USE weather")

In [20]:
cass.execute("CREATE TYPE station_record (tmin int, tmax int)")

InvalidRequest: Error from server: code=2200 [Invalid query] message="A user type with name 'station_record' already exists"

In [23]:
cass.execute("""
create table stations(
    id TEXT,
    name TEXT STATIC,
    date DATE,
    record weather.station_record,
    PRIMARY KEY ((id), date) 
) WITH CLUSTERING ORDER BY (date ASC)
""")

In [28]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"